In this script I will create the following alternative forecast data:

- Naïve Forecast (No-Change)
- Naïve Forecast (Constant Growth)
- ARIMA Model Forecasts
- Forward Rate Forecast

# Naïve Forecast (No-Change)
This is the simplest type of forecasts. Here, I simply predict that the interest rate of prevailing at the end of the previous quartet will correspond to the interest rate prevailing this quarter.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#Two Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TUQE.csv")
TUY=pd.read_csv(relative_path,index_col=0)
#Ten Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TYQE.csv")
TYY=pd.read_csv(relative_path,index_col=0)

In [3]:
#Create different dataframes for different forecast horizons
TUY3=TUY.copy()
TUY6=TUY.copy()
TUY12=TUY.copy()

TYY3=TYY.copy()
TYY6=TYY.copy()
TYY12=TYY.copy()

In [4]:
TUY3["No-Change(F)"]=TUY["2Y"].shift(1)
TUY6["No-Change(F)"]=TUY["2Y"].shift(2)
TUY12["No-Change(F)"]=TUY["2Y"].shift(4)
TYY3["No-Change(F)"]=TYY["10Y"].shift(1)
TYY6["No-Change(F)"]=TYY["10Y"].shift(2)
TYY12["No-Change(F)"]=TYY["10Y"].shift(4)

In [ ]:
TUY3

# Naïve Forecast (Constant Growth)
In this forecast, we assume that the previous quarter to quarter growth will also prevail this quarter. This means that the values are predicted to change, but the growth rate is assumed to stay the same. Leitch (1987, p. 43) calcualted the forefast by: "first calculating a simple growth rate in the interest rate from the period just past to the current period, and then extrapolating that rate of change ahead".

In [6]:
#One-Quarter Ahead Forecast
def const_g1(df,maturity):
    #check wheter value changed as predicted
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            df["No-Change(g)"]=np.nan
            index_1=index
        elif i==2:
            index_2=index
        elif i>2:
            #calculate growth rate
            g=df.loc[index_2,maturity]/df.loc[index_1,maturity]
            #Make Forecast
            Forecast=g*df.loc[index_2,maturity]
            df.loc[index,"No-Change(g)"]=Forecast
            #update previous index value
            index_1=index_2
            index_2=index

In [7]:
#Two-Quarter Ahead Forecast
def const_g2(df,maturity):
    #check wheter value changed as predicted
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            df["No-Change(g)"]=np.nan
            index_1=index
        elif i==2:
            index_2=index
        elif i==3:
            index_3=index
        elif i>3:
            #calculate growth rate
            g=df.loc[index_2,maturity]/df.loc[index_1,maturity]
            #Make Forecast
            Forecast=g*g*df.loc[index_2,maturity]
            df.loc[index,"No-Change(g)"]=Forecast
            #update previous index value
            index_1=index_2
            index_2=index_3
            index_3=index

In [8]:
#Four-Quarter Ahead Forecast
def const_g4(df,maturity):
    #check wheter value changed as predicted
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            df["No-Change(g)"]=np.nan
            index_1=index
        elif i==2:
            index_2=index
        elif i==3:
            index_3=index
        elif i==4:
            index_4=index
        elif i==5:
            index_5=index
        elif i>5:
            #calculate growth rate
            g=df.loc[index_2,maturity]/df.loc[index_1,maturity]
            #Make Forecast
            Forecast=g**4*df.loc[index_2,maturity]
            df.loc[index,"No-Change(g)"]=Forecast
            #update previous index value
            index_1=index_2
            index_2=index_3
            index_3=index_4
            index_4=index_5
            index_5=index

In [9]:
#Apply Function to make forecasts
const_g1(TUY3,"2Y")
const_g1(TYY3,"10Y")
const_g2(TUY6,"2Y")
const_g2(TYY6,"10Y")
const_g4(TUY12,"2Y")
const_g4(TYY12,"10Y")

In [ ]:
TUY3

# ARIMA Model Forecast
To estimate the ARIMA model, we first need to check wheter the time-series of interest rates is stationary and need to take differences if this is not the case.

In [11]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib import pyplot as plt

In [ ]:
plot_acf(TYY["10Y"])

In [ ]:
f=plt.figure()
ax1=f.add_subplot(121)
ax1.set_title("1st Order Differencing")
ax1.plot(TYY["10Y"].diff())
ax2=f.add_subplot(122)
plot_acf(TYY["10Y"].diff().dropna(),ax=ax2)
plt.show()

In [14]:
def n_diff(df_column):
    for i in range (0,4):
        #prepare how many time to difference
        result=adfuller(df_column.diff(i).dropna())
        if result[1]<0.05:
            print("p-value: ", result[1], " achieved with ", i, " times differencing")
            break
    results=[i, result[1]]
    return(results)

In [ ]:
n_diff(TUY.loc["1990-03-31":"1999-12-31","2Y"])
n_diff(TYY.loc["1990-03-31":"1999-12-31","10Y"])

Both of the time-series exhibited non-stationarity; however, stationarity was achieved following a one-time differencing transformation. Next, I will estimate different ARIMA models for both the two-year and ten-year yield. I will estimate it by using data that will not be used for the forecasts, i.e. quarterly data from the 1990s. I will then make the forecasts with the data from Q1 2000. The respective model will be chosen by estimating the AIC, BIC, and HQIC, and take the ones with the lowest values.

In [ ]:
#empty dictionary to store information criteria
inf_crit={}
#iterate through different values of p and q
for p in range (0,5):
    for q in range(0,5):
        #estimate and fit the model
        arima_model=ARIMA(TUY.loc["1990-03-31":"1999-12-31","2Y"],order=(p,1,q))
        model=arima_model.fit()
        #store the information criterion in the dictionary
        order=f"({p},1,{q})"
        inf_crit[order]=[model.bic,model.aic,model.hqic]
        #print(model.summary())
#make a data frame of all information criterions
ic_df=pd.DataFrame(inf_crit).T
ic_df.columns=["BIC","AIC","HQIC"]
#for each criterion get model with lowest value
ic_df.idxmin()
#save information criterion in excel
relative_path = os.path.join('..', '..',"Data",'Results',"StatisticalTests", "Information Criterion", "IC_ARIMA_2Y.xlsx")
ic_df.to_excel(relative_path)

In [ ]:
#empty dictionary to store information criteria
inf_crit={}
#iterate through different values of p and q
for p in range (0,5):
    for q in range(0,5):
        #estimate and fit the model
        arima_model=ARIMA(TYY.loc["1990-03-31":"1999-12-31","10Y"],order=(p,1,q))
        model=arima_model.fit()
        #store the information criterion in the dictionary
        order=f"({p},1,{q})"
        inf_crit[order]=[model.bic,model.aic,model.hqic]
        #print(model.summary())
#make a data frame of all information criterions
ic_df=pd.DataFrame(inf_crit).T
ic_df.columns=["BIC","AIC","HQIC"]
#for each criterion get model with lowest value
ic_df.idxmin()
#save information criterion in excel
relative_path = os.path.join('..', '..',"Data",'Results',"StatisticalTests", "Information Criterion", "IC_ARIMA_10Y.xlsx")
ic_df.to_excel(relative_path)

In [ ]:
ic_df

For the two-year yield, all information criterion indicated that the ideal model is an I(1) model. For the 10-year model, the BIC and HQIC indicate that a I(1) model is appropriate, while the AIC indicates that an ARIMA(4,1,2) model is appropriate. As both the HQIC and BIC indicate a I(1) model, and as the BIC penalizes complexity more than the AIC, consistent with the parsimony principle (Diebold, 2001) I will choose an I(1) model here as well. Now, as forecasts based on an I(1) model essentially are no-change forecasts, I will not implement ARIMA forecasts seperately, but will just use the no-change forecasts.
# Markets Forecast (Future implied Yield)
Here, I take the yield implied the front-month futures contract as forecast.

In [19]:
#Quarterly Implied Yield Data
#10Y
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TYc1QE.csv")
TYc1=pd.read_csv(relative_path,index_col=0)
TYc1=TYc1.dropna()
#2Y
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TUc1QE.csv")
TUc1=pd.read_csv(relative_path,index_col=0)
TUc1=TUc1.dropna()
#10Yc2
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TYc2QE.csv")
TYc2=pd.read_csv(relative_path,index_col=0)
TYc2=TYc2.dropna()
#2Yc2
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Futures", "TUc2QE.csv")
TUc2=pd.read_csv(relative_path,index_col=0)
TUc2=TUc2.dropna()

In [ ]:
TUY3

In [ ]:
gangnam=TUY3.merge(TUc1,how="inner",left_index=True, right_index=True)
gangnam["Implied Yield"]=gangnam["Implied Yield"].shift(1)
gangnam

In [22]:
def iY_merge(forecasts, IY,n_periods):
    forecasts=forecasts.merge(IY,how="inner",left_index=True, right_index=True)
    forecasts["Implied Yield"]=forecasts["Implied Yield"].shift(n_periods)
    return(forecasts)

In [23]:
#Apply Function to make forecasts
TUY3=iY_merge(TUY3,TUc1,1)
TYY3=iY_merge(TYY3,TYc1,1)
TUY6=iY_merge(TUY6,TUc2,2)
TYY6=iY_merge(TYY6,TYc2,2)
TUY12=iY_merge(TUY12,TUc1,4)
TYY12=iY_merge(TYY12,TYc1,4)

In [ ]:
TYY6

# Forward Rate Forecast
To calculate the Forward Rate Forecast, we use the following formula:
$$
F_{m,n} = \sqrt[m]{\frac{(1 + r_{m+n})^{m+n}}{(1 + r_n)^n}} - 1
$$
, where *m* refers to the number of years to maturity for the selected treasury note, *n* refers to the forecast horizon, and *F* refers to the Forward rate. The spot rates which are not directly observable on the market will be estimated using linear interpolation.

In [25]:
#Historical Yield Curve
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "YieldCurve.csv")
YieldCurve=pd.read_csv(relative_path,index_col=0)
YieldCurve=YieldCurve.loc["1990-03-31":YieldCurve.index[-1]]

In [26]:
def forward_rate(m,n,date_index,interp_a,interp_b,interp_c,interp_d,r_n):
    r_mn=np.interp(m+n, [interp_a,interp_b], [YieldCurve.loc[date_index,interp_c],YieldCurve.loc[date_index,interp_d]])
    numerator=(1+r_mn/100)**(m+n)
    denominator=(1+YieldCurve.loc[date_index,r_n]/100)**n
    forward_rate=((numerator/denominator)**(1/m)-1)*100
    return (forward_rate)

In [27]:
#works better if we do not include forward implied yield
def forward_calc(df,m,n,interp_a,interp_b,interp_c,interp_d,r_n,q):
    df["Forward"]=np.nan
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            index_1=index
        else:
            #calculate forward rate
            fr=forward_rate(m,n,index_1,interp_a,interp_b,interp_c,interp_d,r_n)
            #Make Forecast
            df.loc[index,"Forward"]=fr
            #update previous index value
            index_1=index
    #shift forecast according to how many quarters ahead we look at        
    df["Forward"]=df["Forward"].shift(q-1)

In [28]:
#works better if we include future implied yield
def forward_calc(df,m,n,interp_a,interp_b,interp_c,interp_d,r_n,q,yieldcurve):
    df["Forward"]=np.nan
    i=0
    for index, row in yieldcurve.iterrows():
        i=i+1
        if i==1:
            index_1=index
        else:
            if index>=df.index[0]:
                #calculate forward rate
                fr=forward_rate(m,n,index_1,interp_a,interp_b,interp_c,interp_d,r_n)
                #Make Forecast
                df.loc[index,"Forward"]=fr
                #update previous index value
                index_1=index
            else:
                index_1=index
    #shift forecast according to how many quarters ahead we look at        
    df["Forward"]=df["Forward"].shift(q-1)

In [29]:
forward_calc(TUY3,2,0.25,2,3,"2 Yr","3 Yr","3 Mo",1,YieldCurve)
forward_calc(TUY6,2,0.5,2,3,"2 Yr","3 Yr","6 Mo",2,YieldCurve)
forward_calc(TUY12,2,1,2,3,"2 Yr","3 Yr","1 Yr",4,YieldCurve)
forward_calc(TYY3,10,0.25,10,20,"10 Yr","20 Yr","3 Mo",1,YieldCurve)
forward_calc(TYY6,10,0.5,10,20,"10 Yr","20 Yr","6 Mo",2,YieldCurve)
forward_calc(TYY12,10,1,10,20,"10 Yr","20 Yr","1 Yr",4,YieldCurve)

In [ ]:
print(TUY3)
print(TUY6)
print(TUY12)
print(TYY3)
print(TYY6)
print(TYY12)

In [31]:
def filesaver(df,docname):
    #Save file using relative path
    relative_path = os.path.join('..', '..',"Data", 'Consolidated',"Forecasts","AlternativeForecasts",docname)
    df.to_csv(relative_path)

In [32]:
filesaver(TUY3.dropna(),"TUY3.csv")
filesaver(TUY6.dropna(),"TUY6.csv")
filesaver(TUY12.dropna(),"TUY12.csv")
filesaver(TYY3.dropna(),"TYY3.csv")
filesaver(TYY6.dropna(),"TYY6.csv")
filesaver(TYY12.dropna(),"TYY12.csv")